In [ ]:
import respiration.dataset as repository

dataset = repository.from_default()

subject = 'Proband05'
scenario = '101_natural_lighting'

In [ ]:
frames, meta = dataset.get_video_bgr(subject, scenario)

In [ ]:
from respiration.extractor.mtts_can import (
    preprocess_video_frames,
    preprocess_frames_original,
)

# resized, normalized = preprocess_frames_original(frames)
resized, normalized = preprocess_video_frames(frames)

In [ ]:
import matplotlib.pyplot as plt

# Plot resized and normalized frames
_, axs = plt.subplots(1, 2, figsize=(15, 5))

axs[0].imshow(resized[1])
axs[0].set_title('Resized Frame')

axs[1].imshow(normalized[1])
axs[1].set_title('Normalized Frame')

In [ ]:
import matplotlib.pyplot as plt

from respiration.extractor.mtts_can import load_model

frame_depth = 10

# The model expects a number of frames that is a multiple of frame_depth
cut_off = (normalized.shape[0] // frame_depth) * frame_depth
input_resized = resized[:cut_off]
input_normalized = normalized[:cut_off]

cut_off, frame_depth

In [ ]:
input_resized.shape

In [ ]:
# import os
# model_checkpoint = os.path.join('..', 'models', 'last_model.keras')

model = load_model(
    # model_checkpoint=model_checkpoint,
    frame_depth=frame_depth,
)
yptest = model.predict(
    (input_resized, input_normalized),
    batch_size=100
)

In [ ]:
import numpy as np

pulse_pred = np.cumsum(yptest[0])
resp_pred = np.cumsum(yptest[1])

In [ ]:
import respiration.preprocessing as sp

pulse_pred = sp.detrend_tarvainen(pulse_pred, 100)
pulse_pred = sp.butterworth_filter(pulse_pred, meta.fps, 0.75, 2.5)

resp_pred_1_detrend = sp.detrend_tarvainen(resp_pred, 100)
resp_pred_1_filtered = sp.butterworth_filter(resp_pred_1_detrend, meta.fps, 0.08, 0.6)

In [ ]:
# Plot the pulse 
plt.plot(pulse_pred)
plt.title('Predicted Pulse')
plt.xlabel('Time')
plt.ylabel('Pulse')
plt.show()

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(20, 20))

# Add some space between the plots
fig.subplots_adjust(hspace=0.5)

axs[0].plot(resp_pred, label='Predicted Respiration (Raw)')
axs[0].set_title('Predicted Respiration (Raw)')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Respiration')
axs[0].legend()

axs[1].plot(resp_pred_1_detrend, label='Predicted Respiration (Detrend)')
axs[1].set_title('Predicted Respiration (Detrend)')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Respiration')
axs[1].legend()

axs[2].plot(resp_pred_1_filtered, label='Predicted Respiration (Filtered)')
axs[2].set_title('Predicted Respiration (Filtered)')
axs[2].set_xlabel('Time')
axs[2].set_ylabel('Respiration')
axs[2].legend()

gt_signal, gt_sample_rate = dataset.get_ground_truth_rr_signal(subject, scenario)
gt_signal = sp.standard_processing(gt_signal, gt_sample_rate, 0.08, 0.6)
gt_signal = sp.resample_signal(gt_signal, len(resp_pred_1_filtered))

axs[3].plot(gt_signal, label='Ground Truth Respiration')
axs[3].set_title('Ground Truth Respiration')
axs[3].set_xlabel('Time')
axs[3].set_ylabel('Respiration')
axs[3].legend()

In [ ]:
resp_pred_2 = sp.standard_processing(resp_pred, meta.fps, 0.08, 0.6)

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(20, 10))

# Add some space between the plots
fig.subplots_adjust(hspace=0.5)

axs[0].plot(resp_pred_1_filtered, label='Predicted Respiration (1)')
axs[0].set_title('Predicted Respiration (Filtered)')
axs[0].set_xlabel('Time')
axs[0].set_ylabel('Respiration')
axs[0].legend()

axs[1].plot(resp_pred_2, label='Predicted Respiration (2)')
axs[1].set_title('Predicted Respiration (Filtered)')
axs[1].set_xlabel('Time')
axs[1].set_ylabel('Respiration')
axs[1].legend()

In [ ]:
from respiration.analysis import FrequencyExtractor

gt_fe = FrequencyExtractor(gt_signal, gt_sample_rate, lowpass=0.08, highpass=0.6)
pd_fe = FrequencyExtractor(resp_pred_2, meta.fps, lowpass=0.08, highpass=0.6)

gt_bpm = gt_fe.frequency_from_nfcp() * 60
pd_bpm = pd_fe.frequency_from_nfcp() * 60

abs(gt_bpm - pd_bpm)

In [ ]:
from scipy.signal import resample
import respiration.analysis as analysis

# Resample the ground truth signal to the length of the predicted signal
resampled_gt = resample(gt_signal, len(resp_pred_2))

# Calculate the mean squared error
mse = analysis.distance_mse(resampled_gt, resp_pred_2)

# Calculate the Euclidean distance
euclidean_distance = analysis.distance_euclidean(resampled_gt, resp_pred_2)

# Calculate the Pearson correlation
p_correlation = analysis.pearson_correlation(resampled_gt, resp_pred_2)

print(f'Mean Squared Error: {mse:.2f}')
print(f'Euclidean Distance: {euclidean_distance:.2f}')
print(f'Pearson Correlation: {p_correlation:.2f}')